# Assignment 4 
`- 652115013 Narongchai Rongthong`

- Use the method discussed in this chapter to find and rank 5 misspelt candidates for a typo word clowd.
    - Explain and choose 5 candidates from https://www.dcode.fr/levenshtein-distanceLinks to an external site. that are each one edit distance away from the word clowd. (1 pt)
- Create p(x|w) table. (1 pt)
- Create p(x|w)p(w) table using COCA, WIKI, and IULA (one each) (0.5 pt each = 1.5 pt in total)
- Bring context into account, but this time uses COCA, WIKI, and IULA (one each) (0.5 pt each = 1.5 pt in total)
    - Make one sentence by yourself to show that corpus become less dominant.

---
Q1 `- Use the method discussed in this chapter to find and rank 5 misspelt candidates for a typo word clowd.`

Candidate generation (COCA)
- Firstly we're utilizing the Corpus of Contemporary English (COCA — https://www.english-corpora.org/coca/) as the reference corpus.
    - As of of 2019, T ~ 1,001,610,938 terms
    - Candidates was taken from https://www.dcode.fr/levenshtein-distance

So we got the frequecies of:
- CLOD = 282
- CLOW = 63
- CLOUD = 28077
- CLOWN = 7190
- CROWD = 56835

`Now we can start writing code to rank their probability`

In [26]:
import pandas as pd

# Create a DataFrame with words and their frequencies
words = ['clod', 'clow', 'cloud', 'clown', 'crowd']
frequencies = [282, 63, 28077, 7190, 56835]
COCA = pd.DataFrame(list(zip(words, frequencies)), columns=['word', 'frequency'])

# COCA population (As of of 2019, T ~ 1,001,610,938 terms)
COCA_pop = 1001610938 

# Calculate the probability
COCA['P(w)'] = COCA['frequency'] / COCA_pop

# Rank the words by their frequency (highest rank for most frequent)
COCA['rank'] = COCA['frequency'].rank(ascending=False, method='min').astype(int)

COCA

,word,frequency,P(w),rank
0,clod,282,2.815464e-07,4
1,clow,63,6.289867e-08,5
2,cloud,28077,2.803184e-05,2
3,clown,7190,7.178436e-06,3
4,crowd,56835,5.674359e-05,1


*Ranking by frequency reflects the likelihood of each word appearing in the corpus, and thus indicates its likelihood of being the correct word.*

--- 
Q1.1 `- Explain and choose 5 candidates from https://www.dcode.fr/levenshtein-distanceLinks to an external site. that are each one edit distance away from the word clowd. (1 pt)`

Query result from the site

<div>
    <img src="src/image.png" width="500"/>
</div>

<b> This shows these 5 candidates that are one edits away from the word `CLOWD` </b>

    - CLOD	1
    - CLOW	1
    - CLOUD	1
    - CLOWN	1
    - CROWD	1

These words are considered one edit away from CLOWD because they can be transformed into CLOWD through a single edit operation. 

- The typical edit operations are:
    - Insertion: Adding a single character.
    - Deletion: Removing a single character.
    - Substitution: Replacing a single character with another.
    - Transposition (optional in some spell-correction algorithms): Swapping two adjacent characters.

<b>So the operations for each words would be</b>

`CLOWD -> CLOD:`

- Deletion: Remove the letter W.

`CLOWD -> CLOW:`

- Deletion: Remove the letter D.

`CLOWD -> CLOUD:`

- Substitution: Replace the letter W in CLOWD with U.

`CLOWD -> CLOWN:`

- Substitution: Replace the letter D in CLOWD with N.

`CLOWD -> CROWD:`

- Substitution: Replace the letter L in CLOWD with R.

---
Q2 `- Create p(x|w) table. (1 pt)`

To do this we consult the collected list of errors, e.g., Peter Norvig’s collections http://norvig.com/ngrams `/count_1edit.txt`<br>
- so we can find more probable correct spelling based on the common letter misspelled

*Note we cannot model unseen errors*

In [2]:
norvig = pd.read_csv('http://norvig.com/ngrams/count_1edit.txt', sep='\t',encoding = "ISO-8859-1", header=None)
norvig.columns = ['term', 'edit']
norvig = norvig.set_index('term')
print(norvig.head())

      edit
term      
e|i    917
a|e    856
i|e    771
e|a    749
a|i    559


Next we get (count_big.txt) from Peter Norvig’s collection at http://norvig.com/ngrams `/count_big.txt`<br>
This allows us to determine the prior probability of a word's occurrence in general usage.

- This file contains unigram term frequencies (single-word occurrences) from a large corpus of text.
- The data provides the frequency (freq) of each term (term) in the corpus.

*Rare words may have less reliable estimates.*

In [3]:
norvig_orig = pd.read_csv('http://norvig.com/ngrams/count_big.txt',sep='\t', encoding = "ISO-8859-1", header=None)
norvig_orig = norvig_orig.dropna()
norvig_orig.columns=['term','freq']
print(norvig_orig.head())

    term   freq
0      a  21160
1    aah      1
2  aaron      5
3     ab      2
4  aback      3


Now with that data, we can calculate the p(x|w)

In [6]:
from multiprocessing.pool import ThreadPool as Pool
from string import ascii_lowercase
import pandas as pd
import itertools

# Function to calculate the count of characters
def get_count(c):
    global shared_df
    return shared_df.apply(lambda x: x.term.count(c) * x.freq, axis=1).sum()

# Function to initialize the shared dataframe in the pool
def init_pool(df):
    global shared_df
    shared_df = df

# Character set generation
character_set = (
    list(map(''.join, itertools.product(ascii_lowercase, repeat=1))) +
    list(map(''.join, itertools.product(ascii_lowercase, repeat=2)))
)

# Number of cores for parallel processing
core_count = 4

# Creating a frequency DataFrame for the characters
with Pool(core_count, initializer=init_pool, initargs=(norvig_orig,)) as pool:
    freq_list = pool.map(get_count, character_set)

freq_df = pd.DataFrame({'char': character_set, 'freq': freq_list}).set_index('char')


In [28]:
# Adding P(x|w) to COCA DataFrame (use zero instead if doesnt have)
COCA['P(x|w)'] = [
    (norvig.loc['w| '].values[0] if 'w| ' in norvig.index else 0) / freq_df.loc['w'].values[0],  # For "clod"
    (norvig.loc['d| '].values[0] if 'd| ' in norvig.index else 0) / freq_df.loc['d'].values[0],  # For "clow"
    (norvig.loc['o|u'].values[0] if 'o|u' in norvig.index else 0) / freq_df.loc['u'].values[0],  # For "cloud"
    (norvig.loc['d|n'].values[0] if 'd|n' in norvig.index else 0) / freq_df.loc['n'].values[0],  # For "clown"
    (norvig.loc['l|r'].values[0] if 'l|r' in norvig.index else 0) / freq_df.loc['r'].values[0]   # For "crowd"
]
COCA

,word,frequency,P(w),rank,P(x|w)
0,clod,282,2.815464e-07,4,0.000010
1,clow,63,6.289867e-08,5,0.000000
2,cloud,28077,2.803184e-05,2,0.001233
3,clown,7190,7.178436e-06,3,0.000043
4,crowd,56835,5.674359e-05,1,0.000068


---
Q3 `- Create p(x|w)p(w) table using COCA, WIKI, and IULA (one each) (0.5 pt each = 1.5 pt in total)`

We can easily add one for COCA just like so.

In [21]:
# Adding P(x|w)P(w)
COCA['109 P(x|w)P(w)'] = 1e9 * COCA['P(w)'] * COCA['P(x|w)']
COCA

,word,frequency,P(w),rank,P(x|w),109 P(x|w)P(w)
0,CLOD,282,2.815464e-07,4,0.000010,0.002792
1,CLOW,63,6.289867e-08,5,0.000000,0.000000
2,CLOUD,28077,2.803184e-05,2,0.001233,34.555819
3,CLOWN,7190,7.178436e-06,3,0.000043,0.311269
4,CROWD,56835,5.674359e-05,1,0.000068,3.849571


Next we have to find WIKI and IULA, firstly we have to get the data

We can get WIKI data from the website https://www.english-corpora.org/wiki/

In [32]:
# !!! CROWD IS 0 RN DUE TO QUERY LIMIT PLEASE DONT FORGET PLSLSPSLPSLSPls
# Create a DataFrame with words and their frequencies
words = ['clod', 'clow', 'cloud', 'clown', 'crowd']
frequencies = [226, 511, 36352, 11029, 0]  # NOW Using the WIKI frequencies
WIKI = pd.DataFrame(list(zip(words, frequencies)), columns=['word', 'frequency'])

# WIKI population (T ~ 1,900,000,000)
WIKI_pop = 1.9e9

# Calculate the probability
WIKI['P(w)'] = WIKI['frequency'] / WIKI_pop

# Rank the words by their frequency (highest rank for most frequent)
WIKI['rank'] = WIKI['frequency'].rank(ascending=False, method='min').astype(int)

WIKI


,word,frequency,P(w),rank
0,clod,226,1.189474e-07,4
1,clow,511,2.689474e-07,3
2,cloud,36352,1.913263e-05,1
3,clown,11029,5.804737e-06,2
4,crowd,0,0.000000e+00,5


I'll be using IULA that we've downloaded back in handout.

---
Q4 : 
- `Bring context into account, but this time uses COCA, WIKI, and IULA (one each) (0.5 pt each = 1.5 pt in total)`
    - `Make one sentence by yourself to show that corpus become less dominant.`    
